In [12]:
import cv2
import SUNRGBD
import random as rand
import pandas as pd
import numpy as np
import json
import os
import shutil
import sys
import h5py
import scipy.io
import matplotlib.pyplot as plt

import numpy as np
import skimage.draw
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [14]:
print(COCO_WEIGHTS_PATH)

/home/hmeng/Mask_RCNN-for-SUN-RGB-D/mask_rcnn_coco.h5


In [2]:
img_root = "/datasets/home/81/381/hmeng/SUNRGBD/"

In [3]:
event_root = "../../logs/sun20200525T1457"
event_name = "events.out.tfevents.1590443878.dsmlp-jupyter-hmeng"

In [9]:
class SunConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "sun"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 13  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.7
    
    #LEARNING_RATE = 0.001
    LEARNING_RATE = 0.001
    
def detect_and_color_splash(model, image_path=None, video_path=None, image_name=None):
    assert image_path or video_path

    # Image or video?
    if image_path:
        # Run model detection and generate the color splash effect
        print("Running on {}".format(args.image))
        # Read image
        image = skimage.io.imread(args.image)
        # Detect objects
        r = model.detect([image], verbose=1)[0]
        print("r shape")
        print(r['class_ids'])
        print(r['masks'].shape)
        with open("./detect/mask_" + image_name + ".txt", "w") as p_file:
            for i in range(len(r['class_ids'])):
                data_slice = r['masks'][:, :, i]
                print(np.sum(data_slice))
                #np.savetxt(p_file, data_slice.astype(float))
                
class InferenceConfig(SunConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1

In [ ]:
config = InferenceConfig()
model = modellib.MaskRCNN(mode="inference", config=config, model_dir="test")
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
image = 
image_name = "".join([i for i in args.image if i.isdigit()])
detect_and_color_splash(model, image_path=args.image,
                                video_path=args.video, image_name=image_name)